# Download files from an S3 Bucket using *requests*

This small step by step tutorial will guide you to:

- Create an AWS account
- Create an Amazon S3 bucket
- Download and configure the AWS CLI
- Make public the files in the bucket
- Upload your files
- Download the files from the bucket


## Create an S3 Bucket

In the search bar, type 'S3', and click on the first option:
<p align="center">
    <img src="images/amazon_search.PNG" width="500"/>
</p>
In the next window, click on 'Create bucket':

<p align="center">
    <img src="images/Create_bucket.PNG" width="500"/>
</p>

Set a name for your bucket, and choose a region, any from US usually works fine, but make sure to use the same region in the next steps.


# Create an IAM user 

In the search bar look for 'IAM' and click the first option:

<p align="center">
    <img src="images/IAM.PNG" width="500"/>
</p>

Next, click User in the left-hand side, and then click 'Add User'

<p align="center">
    <img src="images/IAM_User.PNG" width="500"/>
</p>

Then fill the user name with the name you want, tick programmatic access, and click Next

In the permissions page, select Attach existing policies directly, tick the AdministratorAccess and then click Next:

<p align="center">
    <img src="images/Policies.PNG" width="500"/>
</p>

On the next pages, simply click Next and create the user. You will see the next page. This page contains your credentials for connecting to your S3 bucket. These credentials will only show once, so make sure to download the .csv file:

<p align="center">
    <img src="images/Credentials.PNG" width="500"/>
</p>



## Download and configure AWS CLI

If you are using Python, you should be familiar with the pip installer. Let's install awscli using:
`pip install awscli`

Next, in the terminal type `aws configure`
Enter the information as it appears in the .csv file you downloaded in the previous step. 

When you are asked about the region name, go to your S3 bucket and look at the AWS Region of your bucket. The region name looks something like 'us-east-1'

When asked about the output format, you can skip this info by pressing enter.

Now, your computer is ready to use boto3

Test that your installation is working by using `aws s3 ls`. You should see something like this:

<p align="center">
    <img src="images/AWSCLI_ls.png" width="500"/>
</p>


# Using boto3 for using your AWS resources from Python

boto3 is a library that allows us to work with AWS from our python script. In this example we are going to simply upload, download and explore S3 buckets, but you can use it to manage other resources such as `EC2`, `RDS`, and `DynamoDB`.

First of all, install boto3 by typing in the terminal `pip install boto3`. Take into account that, in order to use `boto3` you need to have aws configures. 

Let's start by telling to boto3 that we want to use an S3 bucket:

In [1]:

import boto3 
s3_client = boto3.client('s3')



Now, let's upload something to your bucket:

In [2]:
response = s3_client.upload_file('cat-pic.jpg', 'nishaalbucket', 'cat_pic')


*file_name* is the directory of the file you want to upload, *bucket* is the name of your S3 bucket, and *object_name* is the name you want to give to your file once uploaded


Try it yourself!

Now, let's see the content of the bucket:

In [3]:
import boto3
s3 = boto3.resource('s3')

my_bucket = s3.Bucket('nishaalbucket')

for file in my_bucket.objects.all():
    print(file.key)

cat_pic




Once you know the content of it, you can download the files:

In [ ]:

s3 = boto3.client('s3')

s3.download_fileobj(bucket, file_name, object_name)



# Make public the files in the bucket

In your S3 buket, disable the 'Block all public access' option:

<p align="center">
    <img src="images/disable.PNG" width="500"/>
</p>

Once you created it, you can access to it in the bucket list, now you just need to make it public.


To make the objects public, go to http://awspolicygen.s3.amazonaws.com/policygen.html, which will help you create the necessary policy.<br>
- In 'Select Type of Policy' select S3 Bucket Policy. 
- In 'Principal' type ' * '
- In 'Actions' select 'Get Object'
- In 'Amazon Resource Name (ARN)' type arn:aws:s3:::{your_bucket_name}/*
- Press Statement
- Press Generate Policy and copy the text

<p align="center">
    <img src="images/Policy.PNG" width="500"/>
</p>

Go back to your bucket and go to the Permissions tab. In 'Bucket Policy' click Edit. Paste the text you copied and save changes.<br> 
Now your bucket is publicly accesible, and anyone can download your files. 

In your bucket, select the file you want to download, and copy the Object URL.

<p align="center">
    <img src="images/URL.PNG" width="500"/>
</p>

Open a python editor or notebook and use the requests library to download the image from the URL you just copied. Something like this:

In [ ]:
import requests
# Change this with your URL
url = {YOUR URL}
response = requests.get(url)
with open('image.png', 'wb') as f:
    f.write(response.content)



And that's it! you should be able to see the file in the same working directory.

# Challenges
## 1. Cat Scraper

1. Create an AWS S3 bucket (If you did it during the lesson, keep that one)
2. Set the configuration of your AWS CLI
3. In a python script, establish a client connection to the S3 bucket 
4. Download images of cute cats using Selenium
    - Use Selenium to go to the following page: 'https://all-free-download.com/free-photos/cute-cat-jpg.html'
    - Extract the 'src' attribute of each image in the first page
    - With a context manager, create a temporary directory
    - Inside the temporary directory, download all images from all the 'src' you extracted. Use urllib.request.urlretrieve for that purpose. 
    - Still, in the context manager, use the `upload_file` method from the instance you created in step 3
5. Check that the images are not in your local computer, and that they are effectively, in your S3 bucket
6. Once you are finished, save your cat scraper in a module (simply copy the code in a script), and name it `cat_scraper.py`. We will use this script in the next lesson.

In [10]:
from selenium import webdriver
import urllib.request
import tempfile
from tqdm import tqdm
import boto3 

#%%
# Go to the cute cats page
ROOT = 'https://all-free-download.com/free-photos/cute-cat-jpg.html'
driver = webdriver.Chrome('/Users/nishaalajmera/Documents/chromedriver')
driver.get(ROOT)

# Get the list of cat images
elem = driver.find_element_by_class_name('imgcontainer')
if elem:
    img_list = elem.find_elements_by_class_name('img-responsive')
    if img_list:
        src_list = []
        for src in img_list:
            try:
                src_list.append(src.get_attribute('src'))
            except:
                print('No source found')
        # Create a temporary directory, so you don't store images in your local machine
        with tempfile.TemporaryDirectory() as temp_dir:
            for i, scr in enumerate(tqdm(src_list)):
                s3_client = boto3.client('s3')
                response = s3_client.upload_file(scr, 'nishaalbucket', f'cat_{i}.png')

### Quit the driver
elem.close()

  0%|          | 0/68 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'https://images.all-free-download.com/images/graphicthumb/cute_cat_514193.jpg'